setup

In [1]:
import json
import pandas as pd
import json
import get_audio_features as gaf
import get_token as gt 
from IPython.display import (display, HTML)
from requests import post, get

token = gt.get_token()

import json file as a dataframe

In [2]:
tracks_df = pd.read_json('tracks_w_ids.json')
print(len(tracks_df))
tracks_df.head()

6416


,endTime,artistName,trackName,msPlayed,trackId
0,2023-03-18 21:28,Eudes Alvarez,San Rafael de Mi Pueblo,16688,7A80n2HvAs1rZNNs64NN8F
1,2023-03-18 23:52,Sam Cooke,Another Saturday Night,153052,5z8DiKSG4EwxmO6gUi12rZ
2,2023-03-19 19:33,Zoom Soon Bao,Reminiscencia - Remix,2365,0
3,2023-03-19 19:34,Soledad Bravo,Violín de becho,13552,7hkyFS309jQBrwrbxGsJuH
4,2023-03-19 19:37,Soledad Bravo,Violín de becho,196288,7hkyFS309jQBrwrbxGsJuH


In [18]:
tracks_df[tracks_df['trackId'] == "45AFfzJCKIZJLyBlWyQm5s"]

,endTime,artistName,trackName,msPlayed,trackId,streamingDate
205,2023-03-31 19:12:00,Beedle The Bardcore,Stronger - Medieval Bardcore Version,12492,45AFfzJCKIZJLyBlWyQm5s,2023-03-31


change data type of endtime column to datetime64 so I can then format the date as needed (TRANSFORMATION)

In [3]:
tracks_df['endTime'] = pd.to_datetime(tracks_df['endTime'])
tracks_df['streamingDate'] = tracks_df['endTime'].dt.strftime('%Y-%m-%d')
tracks_df.head(10)

,endTime,artistName,trackName,msPlayed,trackId,streamingDate
0,2023-03-18 21:28:00,Eudes Alvarez,San Rafael de Mi Pueblo,16688,7A80n2HvAs1rZNNs64NN8F,2023-03-18
1,2023-03-18 23:52:00,Sam Cooke,Another Saturday Night,153052,5z8DiKSG4EwxmO6gUi12rZ,2023-03-18
2,2023-03-19 19:33:00,Zoom Soon Bao,Reminiscencia - Remix,2365,0,2023-03-19
3,2023-03-19 19:34:00,Soledad Bravo,Violín de becho,13552,7hkyFS309jQBrwrbxGsJuH,2023-03-19
4,2023-03-19 19:37:00,Soledad Bravo,Violín de becho,196288,7hkyFS309jQBrwrbxGsJuH,2023-03-19
5,2023-03-19 19:39:00,Soledad Bravo,Canción para mi américa,113226,54a3QojIm00P4MXL02BZvC,2023-03-19
6,2023-03-19 19:44:00,Soledad Bravo,Pajarillo Verde,59100,1ebslsQcXFPNRBLsa76qEK,2023-03-19
7,2023-03-21 20:26:00,Programming Music,Programming Music for Fast Minds,17809,41F8XFf9aYq1vCMqyuLNFl,2023-03-21
8,2023-03-21 20:26:00,Programming and Coding Jazz,Mellow Moods for Spring Break Getaways,7616,6OzRUo7b23kXFXHWLpigCj,2023-03-21
9,2023-03-21 20:26:00,Programming and Coding Music Club,LoFi Piano Beats for Programming and Coding,4226,61liR3eWrA8BTeUnltA32e,2023-03-21


This was to check how many songs were listened to each day to see if I should make the batches by date.  Because some days have more than 100 songs streamed, I decided to break up the data set in batches of 100 records each.

In [ ]:
from IPython.display import (display, HTML)
batches = tracks_df.groupby(['streamingDate'], as_index=False)['trackName'].count()
display(HTML(batches.to_html()))

remove rows with zero values in the trackId column

In [5]:
tracks_df2 = tracks_df[tracks_df['trackId'] != 0]
print(len(tracks_df2))
tracks_df2.head(10)

6157


,endTime,artistName,trackName,msPlayed,trackId,streamingDate
0,2023-03-18 21:28:00,Eudes Alvarez,San Rafael de Mi Pueblo,16688,7A80n2HvAs1rZNNs64NN8F,2023-03-18
1,2023-03-18 23:52:00,Sam Cooke,Another Saturday Night,153052,5z8DiKSG4EwxmO6gUi12rZ,2023-03-18
3,2023-03-19 19:34:00,Soledad Bravo,Violín de becho,13552,7hkyFS309jQBrwrbxGsJuH,2023-03-19
4,2023-03-19 19:37:00,Soledad Bravo,Violín de becho,196288,7hkyFS309jQBrwrbxGsJuH,2023-03-19
5,2023-03-19 19:39:00,Soledad Bravo,Canción para mi américa,113226,54a3QojIm00P4MXL02BZvC,2023-03-19
6,2023-03-19 19:44:00,Soledad Bravo,Pajarillo Verde,59100,1ebslsQcXFPNRBLsa76qEK,2023-03-19
7,2023-03-21 20:26:00,Programming Music,Programming Music for Fast Minds,17809,41F8XFf9aYq1vCMqyuLNFl,2023-03-21
8,2023-03-21 20:26:00,Programming and Coding Jazz,Mellow Moods for Spring Break Getaways,7616,6OzRUo7b23kXFXHWLpigCj,2023-03-21
9,2023-03-21 20:26:00,Programming and Coding Music Club,LoFi Piano Beats for Programming and Coding,4226,61liR3eWrA8BTeUnltA32e,2023-03-21
10,2023-03-21 20:26:00,Programming Jazz,After Work Relaxation,4353,601ppwJ942tjJ9aDPGYwOT,2023-03-21


checking to see the correct number of rows remain after removing zero values

In [6]:
original_count = 6416
updated_count = 6157
null_trackids = 259
original_count - updated_count == null_trackids

True

### saving this transformed dataframe to json file

In [ ]:
tracks_df2.to_json('tracks_wids_nozeroes.json')

create a dataframe that contains only the track ids, and then creating a list with the numbers you'll use to slice the dataframe to create the batches.

In [7]:
# ids_only_df = tracks_df2['trackId']
ids_only_df = pd.DataFrame(tracks_df2['trackId'])
print(len(ids_only_df))
fixed_batch = []
# for i in range (0, 65):
for i in range (0, 62):
    fixed_batch.append(i*100)
fixed_batch.append(6157)
# fixed_batch.append(6416)
print(fixed_batch)
len(fixed_batch)


6157
[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6157]


63

making sure ids_only_df is in fact a dataframe and not a series.

In [8]:
print(type(ids_only_df))
print(len(ids_only_df))
ids_only_df.head()

<class 'pandas.core.frame.DataFrame'>
6157


,trackId
0,7A80n2HvAs1rZNNs64NN8F
1,5z8DiKSG4EwxmO6gUi12rZ
3,7hkyFS309jQBrwrbxGsJuH
4,7hkyFS309jQBrwrbxGsJuH
5,54a3QojIm00P4MXL02BZvC


This will ultimately create an array containing strings made up of 100 track ids concatenated with a comma as a delimiter.  Each element in the array is one of the batches.  

In [9]:

batch_array = []
# batch_row = []
for j in range (0, 62):
    batch_row = []
    batch_name = "batch"+ str(j)
    start = fixed_batch[j]
    end = fixed_batch[j+1]
    print(f"j = {j} and range for {batch_name} is from {start} to {end}")
    # if i == 67:
    #     batch = ids_only_df.iloc[6400:-1]
    #     batch.to_json(f'batches/{batch_name}.json')
    # else:
    
    # batch = ids_only_df.iloc[start:end]
    batch = ids_only_df.iloc[start:end]
    batch_str = ""
    for index, row in batch.iterrows():
        batch_row.append(row['trackId'])
    delimiter = ","
    batch_str = delimiter.join(batch_row)
    batch_array.append(batch_str)
    # batch.to_json(f'batches/{batch_name}.json')
    # batch.to_csv(f'batches/{batch_name}.csv', index=False)
        
len(batch_array)
    


j = 0 and range for batch0 is from 0 to 100
j = 1 and range for batch1 is from 100 to 200
j = 2 and range for batch2 is from 200 to 300
j = 3 and range for batch3 is from 300 to 400
j = 4 and range for batch4 is from 400 to 500
j = 5 and range for batch5 is from 500 to 600
j = 6 and range for batch6 is from 600 to 700
j = 7 and range for batch7 is from 700 to 800
j = 8 and range for batch8 is from 800 to 900
j = 9 and range for batch9 is from 900 to 1000
j = 10 and range for batch10 is from 1000 to 1100
j = 11 and range for batch11 is from 1100 to 1200
j = 12 and range for batch12 is from 1200 to 1300
j = 13 and range for batch13 is from 1300 to 1400
j = 14 and range for batch14 is from 1400 to 1500
j = 15 and range for batch15 is from 1500 to 1600
j = 16 and range for batch16 is from 1600 to 1700
j = 17 and range for batch17 is from 1700 to 1800
j = 18 and range for batch18 is from 1800 to 1900
j = 19 and range for batch19 is from 1900 to 2000
j = 20 and range for batch20 is from 2000

62

checking the array to make sure everything was concatenated and stored correctly

In [22]:
print(batch_array[9])
test = batch_array[9].split(",")
print(test)
len(test)


3KfbEIOC7YIv90FIfNSZpo,5jgFfDIR6FR0gvlA56Nakr,3BQHpFgAp4l80e1XslIjNI,7iN1s7xHE4ifF5povM6A48,2EqlS6tkEnglzr7tkKAAYD,5ZBeML7Lf3FMEVviTyvi8l,1H7gMYGykdtwZOV6s1N0by,4pbG9SUmWIvsROVLF0zF9s,0pNeVovbiZHkulpGeOx1Gj,1gFNm7cXfG1vSMcxPpSxec,7DD7eSuYSC5xk2ArU62esN,4joiWvli4qJVEW6qZV2i2J,0aym2LBJBk9DAYuHHutrIl,389QX9Q1eUOEZ19vtzzI9O,65vdMBskhx3akkG9vQlSH1,1hTUFqJuQAMjXVGwUPWDqi,01SfTM5nfCou5gQL70r6gs,65vdMBskhx3akkG9vQlSH1,4MLBqAEzNN89o2M9h92Z26,29b2b96jozyD9GPCkOrVLs,5GjPQ0eI7AgmOnADn1EO6Q,1Dg4dFJr3HW7sbA7vPejre,5hhF25sWnZvXhimOHLjTRE,7CZiDzGVjUssMSOXrDNYHL,5By7Pzgl6TMuVJG168VWzS,2B4Y9u4ERAFiMo13XPJyGP,09x9v1o51dbqi5H0u7UGfp,0qsS2A3hwicJCLtcaR6JRQ,3mlMpmY8oZIBFc39D9zLbh,2RnPATK99oGOZygnD2GTO6,1kDkaFlmkdEZiVUogaP9OZ,06ypiqmILMdVeaiErMFA91,1NrbnHlR2BFREcyWXHIHip,2mxByJWOajjiVsLWjNXvDJ,5SUlhldQJtOhUr2GzH5RI7,1hgvykolO4kBIEozATmpyj,25yQPHgC35WNnnOUqFhgVR,1h04XMpzGzmAudoI6VHBgA,4dkoqJrP0L8FXftrMZongF,6tQvjqDIK9GXWIC6mejms8,0ESIjVxnDnCDaTPo6sStHm,68BTFws92cRztMS1oQ7Ewj,7HTH1ppjkkOe7RLoBDKXYJ,7uWFUpGuEfm

100

## Call API to get audio features for each track

create a function that will append json data to an existing json file

In [37]:
def append_features_to_json(new_data, filename='audio_features.json'):
    try:
        # Read existing data from the file
        with open(filename, 'r') as file:
            file_data = json.load(file)
    except FileNotFoundError:
        # If the file doesn't exist yet, create an empty list
        file_data = []
    # Append new_data to the existing data
    file_data.append(new_data)
    # Write the updated data back to the file
    with open(filename, 'w') as file:
        json.dump(file_data, file, indent=4)

test the output of extract_many_audio_features function

In [23]:
features = gaf.extract_many_audio_features(token, batch_array[9])
print(features)
print(len(features))
# print(len(batch_array[9]))
# print(len(batch_array))

length of json_result: 1 length of audio_features[0]: 18
{'danceability': 0.688, 'energy': 0.435, 'key': 9, 'loudness': -11.359, 'mode': 1, 'speechiness': 0.0323, 'acousticness': 0.449, 'instrumentalness': 0, 'liveness': 0.113, 'valence': 0.435, 'tempo': 103.239, 'type': 'audio_features', 'id': '3KfbEIOC7YIv90FIfNSZpo', 'uri': 'spotify:track:3KfbEIOC7YIv90FIfNSZpo', 'track_href': 'https://api.spotify.com/v1/tracks/3KfbEIOC7YIv90FIfNSZpo', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3KfbEIOC7YIv90FIfNSZpo', 'duration_ms': 146333, 'time_signature': 4}
18
2299
62


iterate through array of concatenated ids, send them to function to get their audio features, and append the audio feature json data to a json file

In [ ]:
for ids_string in batch_array:
    features = gaf.extract_many_audio_features(token, ids_string)
    # features2 = features["audio_features"]
    append_features_to_json(features)

In [ ]:
audio_features = pd.read_json('audio_features.json')
# print(type(audio_features))
# afdf = pd.json_normalize(audio_features)
audio_features.head()
# afdf.head()
# print(type(features))
# print(type(features2))

In [ ]:
audio_features.info()

In [39]:
def append_to_json(new_data, filename):
    # try:
    #     # Read existing data from the file
    #     with open(filename, 'a') as file:
    #         file_data = json.load(file)
    # except FileNotFoundError:
    #     # If the file doesn't exist yet, create an empty list
    #     file_data = []
    # # Append new_data to the existing data
    # file_data.append(new_data)
    # # Write the updated data back to the file
    with open(filename, 'a') as file:
        json.dump(new_data, file, indent=4)

## THIS CELL WILL SHOWS HOW TO GET TO EVERY TRACK'S FEATURES

In [40]:
with open('audio_features_v1.json', 'r') as data_file:
    json_stuff = data_file.read()

track_features = json.loads(json_stuff)
# counter = 0
# Access individual items
for track in track_features:
    # if counter == 1:
        # break
    # PAY ATTENTION TO THIS PART HERE!
    for features in track['audio_features']:
        append_to_json(features, 'please_work.json')
        # print(f"\n\nTrack Features:\n{features}")
    # print(f"\n\nTrack Features: \n\n{track['audio_features']}")
    # counter += 1


In [44]:
pleasework = pd.read_json('please_work_copy.json')
pleasework.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6.157000e+03,6157.000000
mean,0.517081,0.432510,5.181582,-12.224055,0.680201,0.054788,0.560518,0.243062,0.172352,0.458772,115.353670,2.122584e+05,3.798766
std,0.185971,0.256007,3.505658,6.520114,0.466436,0.064905,0.337198,0.375634,0.156729,0.293234,31.716705,1.932044e+05,0.594220
min,0.000000,0.000000,0.000000,-44.211000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.151000e+03,0.000000
25%,0.393000,0.221000,2.000000,-14.753000,0.000000,0.032500,0.229000,0.000000,0.094200,0.200000,92.945000,1.540000e+05,4.000000
50%,0.534000,0.416000,5.000000,-10.694000,1.000000,0.039200,0.638000,0.000459,0.113000,0.413000,114.229000,1.964280e+05,4.000000
75%,0.652000,0.633000,8.000000,-7.844000,1.000000,0.051900,0.871000,0.613000,0.181000,0.717000,134.355000,2.440670e+05,4.000000
max,0.949000,0.995000,11.000000,0.291000,1.000000,0.962000,0.996000,1.000000,1.000000,0.997000,227.959000,4.320028e+06,5.000000


In [46]:
pleasework.head(20)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.3750,0.728,9,-10.130,0,0.0433,0.759000,0.860000,0.1410,0.9010,118.414,audio_features,7A80n2HvAs1rZNNs64NN8F,spotify:track:7A80n2HvAs1rZNNs64NN8F,https://api.spotify.com/v1/tracks/7A80n2HvAs1r...,https://api.spotify.com/v1/audio-analysis/7A80...,177800,4
1,0.7760,0.659,9,-7.767,1,0.0510,0.434000,0.000000,0.0868,0.9690,123.094,audio_features,5z8DiKSG4EwxmO6gUi12rZ,spotify:track:5z8DiKSG4EwxmO6gUi12rZ,https://api.spotify.com/v1/tracks/5z8DiKSG4Ewx...,https://api.spotify.com/v1/audio-analysis/5z8D...,160000,4
2,0.4260,0.300,11,-9.717,0,0.0365,0.573000,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,spotify:track:7hkyFS309jQBrwrbxGsJuH,https://api.spotify.com/v1/tracks/7hkyFS309jQB...,https://api.spotify.com/v1/audio-analysis/7hky...,209840,4
3,0.4260,0.300,11,-9.717,0,0.0365,0.573000,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,spotify:track:7hkyFS309jQBrwrbxGsJuH,https://api.spotify.com/v1/tracks/7hkyFS309jQB...,https://api.spotify.com/v1/audio-analysis/7hky...,209840,4
4,0.5550,0.293,3,-8.525,0,0.0470,0.877000,0.000001,0.1180,0.1130,140.935,audio_features,54a3QojIm00P4MXL02BZvC,spotify:track:54a3QojIm00P4MXL02BZvC,https://api.spotify.com/v1/tracks/54a3QojIm00P...,https://api.spotify.com/v1/audio-analysis/54a3...,113227,3
5,0.6140,0.277,9,-12.052,1,0.1200,0.852000,0.000000,0.1060,0.5810,122.659,audio_features,1ebslsQcXFPNRBLsa76qEK,spotify:track:1ebslsQcXFPNRBLsa76qEK,https://api.spotify.com/v1/tracks/1ebslsQcXFPN...,https://api.spotify.com/v1/audio-analysis/1ebs...,180082,4
6,0.1370,0.119,1,-23.390,1,0.0575,0.978000,0.940000,0.1010,0.0374,70.023,audio_features,41F8XFf9aYq1vCMqyuLNFl,spotify:track:41F8XFf9aYq1vCMqyuLNFl,https://api.spotify.com/v1/tracks/41F8XFf9aYq1...,https://api.spotify.com/v1/audio-analysis/41F8...,336001,4
7,0.8180,0.151,9,-21.946,0,0.0718,0.917000,0.917000,0.1110,0.4140,142.153,audio_features,6OzRUo7b23kXFXHWLpigCj,spotify:track:6OzRUo7b23kXFXHWLpigCj,https://api.spotify.com/v1/tracks/6OzRUo7b23kX...,https://api.spotify.com/v1/audio-analysis/6OzR...,121690,4
8,0.5660,0.237,6,-17.012,0,0.0282,0.398000,0.897000,0.1160,0.0761,80.020,audio_features,61liR3eWrA8BTeUnltA32e,spotify:track:61liR3eWrA8BTeUnltA32e,https://api.spotify.com/v1/tracks/61liR3eWrA8B...,https://api.spotify.com/v1/audio-analysis/61li...,147000,4
9,0.6940,0.337,5,-12.300,0,0.0624,0.609000,0.597000,0.1760,0.5690,126.235,audio_features,601ppwJ942tjJ9aDPGYwOT,spotify:track:601ppwJ942tjJ9aDPGYwOT,https://api.spotify.com/v1/tracks/601ppwJ942tj...,https://api.spotify.com/v1/audio-analysis/601p...,190476,4
